# Limpieza de datos – Autos usados en Lima (NeoAuto)

## Contexto
Este proyecto analiza el mercado de autos usados publicados en NeoAuto para la ciudad de Lima.

Los datos fueron obtenidos mediante web scraping con Selenium y corresponden a anuncios activos al momento del scraping.

## Objetivo
Limpiar, estructurar y preparar el dataset para su posterior análisis exploratorio y visualización en Power BI.

## Fuente
- https://neoauto.com

## Fecha de scraping
- 30/01/2026


In [24]:
import pandas as pd
import numpy as np


In [25]:
df = pd.read_csv("Datos_autos_raw.csv")
df.head()


,Marca_Modelo,Precio,Km,Transmision,Combustible,URL
0,Mitsubishi L200 2023,"US$24,000","11,200km",Mecánica,Diesel,https://neoauto.com/auto/usado/mitsubishi-l200...
1,Nissan Qashqai 2015,"US$15,500","83,000km",Automática,Gasolina,https://neoauto.com/auto/usado/nissan-qashqai-...
2,Hyundai Grand I10 2016,"US$10,500","55,000km",Automática,Gasolina,https://neoauto.com/auto/usado/hyundai-grand-i...
3,Bmw X3 30i 2024,"US$52,990","12,800km",Automática - Se...,Gasolina,https://neoauto.com/auto/usado/bmw-x3-30i-2024...
4,Ford Ecosport 2014,"US$12,900","87,781km",Mecánica,Gasolina,https://neoauto.com/auto/usado/ford-ecosport-2...


In [26]:
df.shape

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Marca_Modelo  550 non-null    object
 1   Precio        550 non-null    object
 2   Km            547 non-null    object
 3   Transmision   550 non-null    object
 4   Combustible   550 non-null    object
 5   URL           550 non-null    object
dtypes: object(6)
memory usage: 25.9+ KB


In [27]:
df.isna().sum()


Marca_Modelo    0
Precio          0
Km              3
Transmision     0
Combustible     0
URL             0
dtype: int64

In [28]:
df[df.isnull().any(axis=1)]


,Marca_Modelo,Precio,Km,Transmision,Combustible,URL
67,Chevrolet Corvette 2026,"US$149,000",NaN,Automática - Se...,Gasolina,https://neoauto.com/auto/usado/chevrolet-corve...
291,Toyota Tacoma 2025,"US$125,000",NaN,Automática - Se...,Gasolina,https://neoauto.com/auto/usado/toyota-tacoma-2...
478,Toyota Corolla Cross 2024,"US$20,500",NaN,Automática - Se...,Gasolina,https://neoauto.com/auto/usado/toyota-corolla-...


## Análisis de valores nulos

Se identificaron 3 registros con valores nulos en la variable `Km`.

Tras una revisión manual de los anuncios correspondientes, se determinó que estos vehículos
corresponden a autos nuevos (0 km), los cuales no muestran el kilometraje en la tarjeta de listado
de NeoAuto, pero sí en la ficha detallada del vehículo.

Por lo tanto, estos valores nulos serán tratados como `0 km`.


In [29]:
# Los NaN en Km corresponden a autos nuevos (0 km)
df["Km"] = df["Km"].fillna("0 km")

In [30]:
df.isna().sum()


Marca_Modelo    0
Precio          0
Km              0
Transmision     0
Combustible     0
URL             0
dtype: int64

In [31]:
# Eliminar espacios extra en todas las columnas de texto
for col in df.columns:
    df[col] = df[col].astype(str).str.strip()


In [32]:
df["Precio"] = (
    df["Precio"]
    .str.replace("US$", "", regex=False)
    .str.replace(",", "")
    .str.strip()
)


In [33]:
df["Km"] = (
    df["Km"]
    .str.replace("km", "", case=False)
    .str.replace(",", "")
    .str.strip()
)


### Conversión de variables numéricas

Las columnas `Precio` y `Km` fueron convertidas a tipo numérico (`float`) para permitir
operaciones estadísticas y análisis posteriores.



Durante la conversión de tipos se detectaron valores no numéricos en la columna `Km`.
Tras la revisión manual de los anuncios, se confirmó que estos casos correspondían a
vehículos publicados con **0 km**, por lo que se normalizaron como cero antes de la
conversión a tipo numérico.

In [34]:
# Normalizar valores no numéricos en Km como 0
df["Km"] = pd.to_numeric(df["Km"], errors="coerce").fillna(0)


In [35]:
df["Precio"] = df["Precio"].astype(int)
df["Km"] = df["Km"].astype(int)


In [36]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Marca_Modelo  550 non-null    object
 1   Precio        550 non-null    int32 
 2   Km            550 non-null    int32 
 3   Transmision   550 non-null    object
 4   Combustible   550 non-null    object
 5   URL           550 non-null    object
dtypes: int32(2), object(4)
memory usage: 21.6+ KB


Con los tipos de datos corregidos y sin valores nulos, se procede a revisar la
consistencia de las variables categóricas.


In [37]:
df["Transmision"].value_counts()


Transmision
Automática - Se...    301
Automática            145
Mecánica              104
Name: count, dtype: int64

### Limpieza de la variable Transmisión


In [38]:
df["Transmision"] = df["Transmision"].replace({
    "Automática - Se...": "Automática - Secuencial"
})


Durante el scraping, la categoría **“Automática - Secuencial”** aparece truncada en la tarjeta del anuncio
como **“Automática - Se...”**, debido al diseño visual de la plataforma NeoAuto.

Para mantener la fidelidad al dato original, la extracción se realizó tal como aparece en la tarjeta.
Posteriormente, durante la etapa de limpieza de datos, se estandarizó el valor para unificar la categoría
bajo el nombre completo **“Automática - Secuencial”**

In [39]:
df["Transmision"].value_counts()


Transmision
Automática - Secuencial    301
Automática                 145
Mecánica                   104
Name: count, dtype: int64

### Distribución del tipo de combustible


In [40]:
df["Combustible"].value_counts()


Combustible
Gasolina     459
Diesel        35
Dual          35
Gas GLP        9
Gas GNV        6
Eléctrico      5
Gas            1
Name: count, dtype: int64

La mayoría de los vehículos analizados utilizan gasolina como combustible principal.
Las categorías alternativas (Diesel, Dual, Gas GLP, Gas GNV, Eléctrico) presentan una
frecuencia significativamente menor, lo cual refleja la estructura real del mercado
de autos usados en Lima.

Se identificó una categoría residual denominada “Gas” con una frecuencia mínima (1 registro).
Dado que NeoAuto distingue explícitamente entre Gas GLP y Gas GNV en sus filtros oficiales, este valor fue interpretado como una clasificación ambigua introducida por el anunciante.
Considerando su baja frecuencia y el riesgo de imputación incorrecta, el registro fue conservado sin modificación

In [41]:
df.head()

,Marca_Modelo,Precio,Km,Transmision,Combustible,URL
0,Mitsubishi L200 2023,24000,11200,Mecánica,Diesel,https://neoauto.com/auto/usado/mitsubishi-l200...
1,Nissan Qashqai 2015,15500,83000,Automática,Gasolina,https://neoauto.com/auto/usado/nissan-qashqai-...
2,Hyundai Grand I10 2016,10500,55000,Automática,Gasolina,https://neoauto.com/auto/usado/hyundai-grand-i...
3,Bmw X3 30i 2024,52990,12800,Automática - Secuencial,Gasolina,https://neoauto.com/auto/usado/bmw-x3-30i-2024...
4,Ford Ecosport 2014,12900,87781,Mecánica,Gasolina,https://neoauto.com/auto/usado/ford-ecosport-2...


-------

### Separación de la variable Marca_Modelo

La variable `Marca_Modelo` contiene información compuesta correspondiente a la
marca del vehículo, su modelo y el año de fabricación.

Con el objetivo de habilitar análisis más detallados (precio por marca, modelo
y depreciación por año), esta variable fue descompuesta en tres columnas:

- `Marca`
- `Modelo`
- `Anio`

La extracción se realizó mediante reglas de texto consistentes con el formato
observado en los anuncios de NeoAuto.


In [42]:
# Extraer año (últimos 4 dígitos)
df["Anio"] = df["Marca_Modelo"].str.extract(r"(\d{4})$").astype(int)



In [43]:
# Extraer marca (primera palabra)
df["Marca"] = df["Marca_Modelo"].str.split().str[0]


In [44]:
# Extraer modelo eliminando marca y año
df["Modelo"] = (
    df["Marca_Modelo"]
    .str.replace(r"^\w+\s*", "", regex=True)
    .str.replace(r"\s*\d{4}$", "", regex=True)
    .str.strip()
)


In [45]:
df.head()

,Marca_Modelo,Precio,Km,Transmision,Combustible,URL,Anio,Marca,Modelo
0,Mitsubishi L200 2023,24000,11200,Mecánica,Diesel,https://neoauto.com/auto/usado/mitsubishi-l200...,2023,Mitsubishi,L200
1,Nissan Qashqai 2015,15500,83000,Automática,Gasolina,https://neoauto.com/auto/usado/nissan-qashqai-...,2015,Nissan,Qashqai
2,Hyundai Grand I10 2016,10500,55000,Automática,Gasolina,https://neoauto.com/auto/usado/hyundai-grand-i...,2016,Hyundai,Grand I10
3,Bmw X3 30i 2024,52990,12800,Automática - Secuencial,Gasolina,https://neoauto.com/auto/usado/bmw-x3-30i-2024...,2024,Bmw,X3 30i
4,Ford Ecosport 2014,12900,87781,Mecánica,Gasolina,https://neoauto.com/auto/usado/ford-ecosport-2...,2014,Ford,Ecosport


## Exportación del dataset limpio

El dataset limpio y estructurado se exporta a un archivo CSV
para su uso en análisis exploratorio y visualización en Power BI.


In [46]:
df.to_csv("Datos_autos_limpios.csv", index=False, encoding="utf-8")